# BepiColombo SPICE Kernel Dataset Status

Last updated on {current_time} by Marc Costa Sitja (ESAC/ESA). SKD version: {skd_version}


## Startup

The metakernel is loaded, the scenario covers a week with a finish time set by the end of coverage of the latest Measured Attitude Kernel.

In [ ]:
from spiops import spiops

spiops.load('{metakernel}')               

start_time = '{start_time_measured}'                                      # Start time
finish_time ='{finish_time_measured}'  

interval = spiops.TimeWindow(start_time, finish_time,resolution=60) # spiops object TimeWindow generated
sun = spiops.Target('SUN', time=interval, frame='IAU_SUN')       # spiops object Target Mars generated
mpo = spiops.Observer('MPO', time=interval, target=sun)            # spiops object Observer TGO generated
mtm = spiops.Observer('MTM', time=interval, target=sun)            # spiops object Observer TGO generated

## Coverage

The coverage provided by the SPK and CK files is displayed for the Operational and the Planning meta-kernels

In [ ]:
spiops.ck_coverage_timeline('{metakernel}', 'mpo')

In [ ]:
spiops.spk_coverage_timeline('{metakernel}', 'mpo')

## Attitude Error
Comparison of +Z axis orientation between predicted and measured attitude in milidegrees 

In [ ]:
measured_ck = '{skd_path}/ck/{measured_ck}'
predicted_ck = '{skd_path}/ck/{predicted_ck}'
resolution = 4


spiops.ckdiff_error(measured_ck, predicted_ck, 'MPO_SPACECRAFT', 'J2000', resolution, 0.001, 
                    plot_style='circle', utc_start=start_time, utc_finish=finish_time, notebook=True)

## S/C Structures Orientation
Quaternions, MTM and MPO Solar Array Orientation and Solar Aspect Angle, High Gain Antenna Orientation and HGA boresight-Earth Angle.

In [ ]:
mpo.Plot('quaternions', notebook=True)         # MPO Orientation (quaternions w.r.. J2000)

In [ ]:
mpo.Plot('sa_ang', notebook=True)              # MPO Solar Array (SA) Angles

In [ ]:
mtm.Plot('sa_ang', notebook=True)              # MTM Solar Array (SA) Angles

In [ ]:
mpo.Plot('saa_sa', notebook=True)              # MPO SA Solar Aspect Angle

In [ ]:
mtm.Plot('saa_sa', notebook=True)              # MTM SA Solar Aspect Angle

In [ ]:
mpo.Plot('hga_angles', notebook=True)          # MTM High Gain Antenna Angles

In [ ]:
mpo.Plot('hga_earth', notebook=True)           # High Gain Antenna - Earth Angle

## S/C Clock Drift 

The following plot shows the drift of the S/C Clock.

In [ ]:
mpo.Plot('clock_drift', notebook=True)   